In [38]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [39]:

import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from huggingface_hub import login

from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [40]:
from dotenv import load_dotenv
load_dotenv()

True

In [41]:
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
login(hf_token, add_to_git_credential=True)

In [48]:
AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Llama-3.2-3B-Instruct"
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [49]:
audio_filename = "/Users/sungminhong/Documents/Udemi/LLM_tutorial/3rd_week/denver_extract.mp3"

In [50]:
openai_api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI(api_key=openai_api_key)

In [51]:
audio_file = open(audio_filename, "rb")

In [52]:
audio_file

<_io.BufferedReader name='/Users/sungminhong/Documents/Udemi/LLM_tutorial/3rd_week/denver_extract.mp3'>

In [53]:
speech_to_text = openai.audio.transcriptions.create(
    file=audio_file,
    model=AUDIO_MODEL,
    response_format="text"
)

In [54]:
print(speech_to_text)

and kind of the confluence of this whole idea of the confluence week, the merging of two rivers and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism and you'll hear a little bit more about our confluence week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous People's Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the United Sta

In [55]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{speech_to_text}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [ ]:
# quant_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_quant_type="nf4"
# )
# # CUDA를 사용할때 쓰는 양자화 설정

In [64]:
chat_completion = openai.chat.completions.create(
    model = "gpt-4o-mini",
    messages = messages,
    stream = True
)

In [65]:
for chunk in chat_completion:

    content = chunk.choices[0].delta.content


    if content:
        print(content, end="", flush = True)

print()

# Denver City Council Meeting Minutes

**Date:** October 9, 2017  
**Location:** Denver City Council Chambers  
**Attendees:**  
- Council President  
- Councilman Lopez  
- Councilman Clark  
- Councilman Ortega  
- Councilman Kinneach  
- Councilman Martega  
- Councilman Gilmour  
- Councilman Cashman  
- Councilman Espinosa  
- Councilman New  
- Councilman Sussman  
- City Clerk

## Summary
The Denver City Council convened on October 9, 2017, to discuss the observance of Indigenous Peoples Day and the significance of the newly adopted logo representing the confluence of the Cherry Creek and South Platte Rivers. The meeting included a proclamation recognizing Indigenous peoples' contributions and cultural importance in the Denver area.

## Key Discussion Points
- **Logo Presentation**: The logo symbolizes the merging of rivers and the importance of water resources in the current political context.
- **Indigenous Peoples Day**: The council celebrated the contributions of Indigenous 

In [ ]:
response_message = chat_completion.choices[0].message.content
print(response_message)

# Denver City Council Meeting Minutes

**Date:** October 9, 2017  
**Location:** Denver City Council Chamber

## Attendees
- Council members: Clerk, Espinosa, Flynn, Gilmour, Cashman, Kenneche, Lopez, New, Ortega, Sussman, Mr. President
- Additional attendees included community members and representatives of the Indigenous Peoples.

## Summary
The Denver City Council convened to celebrate Indigenous Peoples Day with a proclamation acknowledging the contributions of Indigenous Peoples to the city and local culture. The meeting included discussions about the significance of water in Indigenous culture and announcements for community events. 

## Discussion Points
1. **Proclamation 1127**: 
   - Read by Councilman Lopez, this proclamation was to observe Indigenous Peoples Day in Denver.
   - It recognized the historical significance of Indigenous Peoples in the region and highlighted various contributions to local culture and society.
   - The proclamation encourages education about Indig

In [ ]:
outputs = model.generate(input, max_new_tokens=2000, streamer=streamer)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(LLAMA, use_fast=False)
# tokenizer.pad_token = tokenizer.eos_token
# input = tokenizer.apply_chat_template(messages,
#                                       return_tensors = 'pt')


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [58]:
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(
    LLAMA,
    torch_dtype=torch.float16,  # float16으로 데이터 타입 지정
    device_map="mps"           # 모델을 MPS 장치에 바로 올림
)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

RuntimeError: Invalid buffer size: 6.72 GB

In [30]:
# 양자화된 모델을 로컬에 저장
print("💾 양자화된 모델을 로컬에 저장합니다...")
model.save_pretrained("./Llama-3.1-8B-Instruct")
tokenizer.save_pretrained("./Llama-3.1-8B-Instruct")
print("✅ 저장 완료!")

💾 양자화된 모델을 로컬에 저장합니다...
✅ 저장 완료!


In [59]:
outputs = model.generate(input, max_new_tokens=2000, streamer=streamer)

NameError: name 'model' is not defined